# Retrieval-Augmented Generation (RAG)

* [Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks](https://arxiv.org/abs/2005.11401)
* [RAG From Scratch](https://github.com/langchain-ai/rag-from-scratch?tab=readme-ov-file)
* [12 RAG Pain Points and Solutions](https://originshq.com/blog/12-rag-pain-points-and-solutions/)
* [Seven Failure Points When Engineering a Retrieval Augmented Generation System](https://arxiv.org/pdf/2401.05856)
* [Архитектура RAG: полный гайд](https://habr.com/ru/amp/publications/791034/)